In [491]:
import sqlalchemy as sa
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots

import plotly.graph_objects as go
import plotly.figure_factory as ff

from matplotlib.animation import FuncAnimation, writers
import numpy as np

In [492]:
SERVER='192.168.0.245'
SQLSERVER_DATABASE='lookingglass'
SQLSERVER_USER='thecaptain'
SQLSERVER_PASSWORD='99Redbal00ns'
PORT=1433
SQLSERVER_BASE_URI = 'mssql+pyodbc://{USER}:{PWD}@{SERVER}:{PORT}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'

engine = sa.create_engine(SQLSERVER_BASE_URI.format(
  SERVER=SERVER,
  DATABASE=SQLSERVER_DATABASE,
  USER=SQLSERVER_USER,
  PWD=SQLSERVER_PASSWORD,
  PORT=PORT
))

In [493]:
match_game_df = pd.read_sql_table(table_name='leaguematchgame', con=engine)
match_round_df = pd.read_sql_table(table_name='leaguematch', con=engine)

In [494]:
round_df = match_round_df[['ngs_match_id', 'match_round','scheduled_start_time', 'match_winner', 'is_forfeit', 'home_team', 'away_team', 'division_concat']]
round_df.scheduled_start_time = np.where(round_df.scheduled_start_time.isnull(), pd.to_datetime('1/31/2022'), round_df.scheduled_start_time)
round_df['scheduled_date'] = pd.to_datetime(round_df['scheduled_start_time']).dt.strftime('%Y-%m-%d')
round_df.drop(['scheduled_start_time'], inplace=True, axis=1)
round_score_df = match_game_df[['ngs_match_id','team_name', 'total_score', 'is_domination']].drop_duplicates()

In [495]:
match_df = pd.merge(round_df, round_score_df, left_on='ngs_match_id', right_on='ngs_match_id', how='left')
match_df['vs_team'] = np.where(match_df.team_name!=match_df.away_team, match_df.away_team, match_df.home_team)
match_df.drop(['home_team', 'away_team'], inplace=True, axis=1)

In [496]:
match_df.total_score.fillna(0, inplace=True)
match_df.drop_duplicates(inplace=True)

match_df.is_forfeit = np.where(match_df.is_forfeit.isnull(), False, match_df.is_forfeit)
match_df.team_name = np.where(( match_df.team_name.isnull() & match_df.is_forfeit == True), match_df.match_winner, match_df.team_name)
match_df.total_score = np.where(( (match_df.team_name==match_df.match_winner) & match_df.is_forfeit == True), 3.0, match_df.total_score)
match_df.is_domination = np.where(( (match_df.team_name==match_df.match_winner) & match_df.is_forfeit == True), True, match_df.is_domination)
match_df.is_domination = np.where(match_df.is_domination == True, 1, 0)
match_df['is_win'] = np.where( match_df.team_name==match_df.match_winner, 1, 0)
match_df['is_loss'] = np.where( match_df.team_name!=match_df.match_winner, 1, 0)
match_df['game_cnt'] = 1
match_df.drop(['ngs_match_id'], inplace=True, axis=1)

In [497]:
match_df['scheduled_week'] = pd.to_datetime(pd.to_datetime(match_df['scheduled_date']) - pd.to_timedelta(pd.to_datetime(match_df['scheduled_date']).dt.weekday % 7, unit='D'))
match_df['week'] = pd.to_datetime(pd.to_datetime(match_df['scheduled_date']) - pd.to_timedelta(pd.to_datetime(match_df['scheduled_date']).dt.weekday % 7, unit='D')).dt.strftime('%a %m-%d')
match_df['week_number'] = pd.to_datetime(match_df['scheduled_date']).dt.isocalendar().week
match_df['score']=match_df.groupby(['scheduled_week', 'team_name'])['total_score'].cumsum()
match_df['dominations']=match_df.groupby(['scheduled_week', 'team_name'])['is_domination'].cumsum()
match_df['wins']=match_df.groupby(['scheduled_week', 'team_name'])['is_win'].cumsum()
match_df['loss']=match_df.groupby(['scheduled_week', 'team_name'])['is_loss'].cumsum()
match_df['games']=match_df.groupby(['scheduled_week', 'team_name'])['game_cnt'].cumsum()
match_df['win_rate']=(match_df.wins / match_df.games) * 100
match_df.sort_values(by='scheduled_week', ascending=True, inplace=True)
match_df = match_df[match_df['team_name'].notnull()]

In [562]:
fig = None
df = None
df = match_df[['week', 'week_number', 'scheduled_week', 'team_name', 'division_concat', 'score']]
df = df.groupby(['team_name','week_number', 'week', 'scheduled_week', 'division_concat'])['score'].max()
x = df.groupby(['team_name']).cumsum(axis=0).reset_index()

piv = x.pivot(index=['team_name'], columns="scheduled_week", values="score")
piv = piv.fillna(0)
# piv.head(100)

# y = pd.crosstab(index=[x.team_name], columns=x.week, values=x.score, aggfunc='sum', margins=True)

In [540]:
divs = [ div for div in x['division_concat'].unique() ]

fig = make_subplots(rows=len(divs), cols=2,row_titles=divs)

for div in divs:
  idx = divs.index(div) + 1
  x_df = x[x['division_concat'] == div]
  max_df = x_df.groupby(['team_name'])['score'].max().reset_index()
  max_score = x_df.score.max() / 2
  a = x_df[x_df.team_name.isin(max_df[max_df['score'] > max_score].team_name.to_list())]
  b = x_df[x_df.team_name.isin(max_df[max_df['score'] < max_score].team_name.to_list())]

  a.set_index('team_name', inplace=True)
  b.set_index('team_name', inplace=True)

  fig1 = px.line(a, x="scheduled_week", y="score", color=a.index.to_list(), markers=True, title=div)
  fig2 = px.line(b, x="scheduled_week", y="score", color=b.index.to_list(), markers=True, title=div)

  for d in fig1.data:
    fig.add_trace(d, row=idx, col=1)

  for d in fig2.data:
    fig.add_trace(d, row=idx, col=2)

fig.layout.update({'height':1000})
fig.show()

In [500]:
fig.write_html("file.html")